In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pathlib import Path
from os import environ
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [2]:
# === Data Handler ===
class DataHandler:
    def __init__(self, batch_size=32):
        self.batch_size = batch_size

    def load_dataset(self, dataset_name, scaler_transform):
        from buildings_bench import load_torch_dataset
        return list(load_torch_dataset(
            dataset_name,
            apply_scaler_transform=scaler_transform,
            scaler_transform_path=Path(environ["TRANSFORM_PATH"])
        ))

    def create_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

In [3]:
class Model(nn.Module):
    DEFAULT_CONTEXT_LEN = 168
    DEFAULT_PRED_LEN = 24

    def __init__(self, activation):
        super().__init__()
        self.context_len = self.DEFAULT_CONTEXT_LEN
        self.pred_len = self.DEFAULT_PRED_LEN
        self.activation = self._get_activation(activation)
        self.embeddings = self._create_embeddings()

    def _create_embeddings(self):
        return nn.ModuleDict({
            'power': nn.Linear(1, 64),
            'building': nn.Embedding(2, 32),
            'lat': nn.Linear(1, 32),
            'lon': nn.Linear(1, 32)
        })

    def _get_activation(self, name):
        return {
            "relu": nn.ReLU(),
            "tanh": nn.Tanh(),
            "gelu": nn.GELU(),
            "leaky_relu": nn.LeakyReLU()
        }.get(name.lower(), nn.ReLU())

    def _data_pre_process(self, x):
        lat = self.embeddings['lat'](x['latitude'])
        lon = self.embeddings['lon'](x['longitude'])
        btype = self.embeddings['building'](x['building_type'].squeeze(-1))
        load = self.embeddings['power'](x['load'])
        return torch.cat([lat, lon, btype, load], dim=2)

class NN(Model):
    def __init__(self, activation):
        super().__init__(activation)
        self.model = self._build_model()

    def _build_model(self):
        input_dim = self.context_len * 160
        return nn.Sequential(
            nn.Linear(input_dim, 512), self.activation,
            nn.Linear(512, 256), self.activation,
            nn.Linear(256, 128), self.activation,
            nn.Linear(128, self.pred_len)
        )

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        x_flat = ts_embed[:, :self.context_len, :].reshape(x['load'].shape[0], -1)
        return self.model(x_flat).unsqueeze(-1)

class RNN(Model):
    def __init__(self, activation="relu"):
        super().__init__(activation)
        self.rnn1, self.rnn2, self.output_layer = self._build_model()

    def _build_model(self):
        rnn1 = nn.RNN(160, 128, batch_first=True)
        rnn2 = nn.RNN(128, 128, batch_first=True)
        output_layer = nn.Linear(128, self.pred_len)
        return rnn1, rnn2, output_layer

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        out1, _ = self.rnn1(ts_embed)
        out2, _ = self.rnn2(out1)
        last_hidden = self.activation(out2[:, -1, :])
        return self.output_layer(last_hidden).unsqueeze(-1)

class LSTM(Model):
    def __init__(self, activation="relu"):
        super().__init__(activation)
        self.lstm1, self.lstm2, self.output_layer = self._build_model()

    def _build_model(self):
        lstm1 = nn.LSTM(160, 128, batch_first=True)
        lstm2 = nn.LSTM(128, 128, batch_first=True)
        output_layer = nn.Linear(128, self.pred_len)
        return lstm1, lstm2, output_layer

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        out1, _ = self.lstm1(ts_embed)
        out2, _ = self.lstm2(out1)
        last_hidden = self.activation(out2[:, -1, :])
        return self.output_layer(last_hidden).unsqueeze(-1)

class GRU(Model):
    def __init__(self, activation="relu"):
        super().__init__(activation)
        self.gru1, self.gru2, self.output_layer = self._build_model()

    def _build_model(self):
        gru1 = nn.GRU(160, 128, batch_first=True)
        gru2 = nn.GRU(128, 128, batch_first=True)
        output_layer = nn.Linear(128, self.pred_len)
        return gru1, gru2, output_layer

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        out1, _ = self.gru1(ts_embed)
        out2, _ = self.gru2(out1)
        last_hidden = self.activation(out2[:, -1, :])
        return self.output_layer(last_hidden).unsqueeze(-1)

class Transformer(Model):
    def __init__(self, device):
        super().__init__()

        if MODEL == "TransformerWithGaussian-L":
            environ["CHECKPOINT_PATH"] = f'{PATH}/checkpoints/Transformer_Gaussian_L.pt'
        environ["CONFIG_PATH"] = f'{PATH}/configs/{MODEL}.toml'

In [4]:
class Trainer:
    def __init__(self, model_name, device, scaler_transform, activation='relu', optimizer_name='adam', lr=1e-3):
        self.model_name = model_name
        self.device = device
        self.scaler_transform = scaler_transform
        self.activation = activation
        self.optimizer_name = optimizer_name
        self.lr = lr
        self.model = self._load_model()
        self.optimizer = self._get_optimizer()
        self.loss_fn = nn.MSELoss()
        self.handler = DataHandler(batch_size=32)

    def _load_model(self):
        model_map = {
            'NN': NN,
            'RNN': RNN,
            'LSTM': LSTM,
            'GRU': GRU
        }
        if self.model_name not in model_map:
            raise ValueError(f"Unsupported model: {self.model_name}")
        return model_map[self.model_name](activation=self.activation).to(self.device)

    def _get_optimizer(self):
        opt_map = {
            'adam': torch.optim.Adam,
            'sgd': torch.optim.SGD,
            'adamw': torch.optim.AdamW
        }
        optimizer_cls = opt_map.get(self.optimizer_name.lower(), torch.optim.Adam)
        return optimizer_cls(self.model.parameters(), lr=self.lr)

    def train(self, train_buildings, epochs=5):
        self.model.train()
        for epoch in range(epochs):
            total_loss = 0.0
            for building_id, building_dataset in train_buildings:
                dataloader = self.handler.create_dataloader(building_dataset)
                for batch in dataloader:
                    for key, value in batch.items():
                        batch[key] = value.to(self.device)
                    self.optimizer.zero_grad()
                    predictions = self.model(batch)
                    targets = batch['load'][:, self.model.context_len:, 0]
                    loss = self.loss_fn(predictions[:, :, 0], targets)
                    loss.backward()
                    self.optimizer.step()
                    total_loss += loss.item()
            print(f"[{self.model_name}] Epoch {epoch + 1}: Loss = {total_loss:.4f}")
        self.model.eval()
        return self.model

    def evaluate(self, test_buildings):
        self.model.eval()
        results = {}
        mae_total = 0.0
        rmse_total = 0.0
        r2_total = 0.0
        count = 0
        for building_id, building_dataset in test_buildings:
            inverse_transform = building_dataset.datasets[0].load_transform.undo_transform
            dataloader = self.handler.create_dataloader(building_dataset)
            target_list = []
            prediction_list = []
            with torch.no_grad():
                for batch in dataloader:
                    for key, value in batch.items():
                        batch[key] = value.to(self.device)
                    predictions = self.model(batch)
                    targets = batch['load'][:, self.model.context_len:]
                    targets = inverse_transform(targets)
                    predictions = inverse_transform(predictions)
                    prediction_list.append(predictions.detach().cpu())
                    target_list.append(targets.detach().cpu())
            predictions_all = torch.cat(prediction_list)
            targets_all = torch.cat(target_list)
            mae = torch.abs(predictions_all - targets_all).mean().item()
            rmse = torch.sqrt(((predictions_all - targets_all) ** 2).mean()).item()
            r2 = 1 - (((predictions_all - targets_all) ** 2).sum() / ((targets_all - targets_all.mean()) ** 2).sum()).item()
            mae_total += mae
            rmse_total += rmse
            r2_total += r2
            count += 1
            results[building_id] = (predictions_all, targets_all)
        return results, mae_total / count, rmse_total / count, r2_total / count


In [ ]:
# === Main Execution ===

# Configuration dictionary
config = {
    "PATH": "/pscratch/sd/n/nrushad",
    "dataset_name": "ideal",
    "scaler_transform": "boxcox",
    "batch_size": 32,
    "epochs": 10,
    "activation": "gelu",             # Options: relu, tanh, leaky_relu, gelu
    "optimizer_name": "adamw",        # Options: adam, sgd, adamw
    "lr": 1e-3
}

# Set environment variables
environ["PATH"] = config["PATH"]
environ["REPO_PATH"] = f"{config['PATH']}/BuildingsBenchTutorial/BuildingsBench/"
environ["BUILDINGS_BENCH"] = f"{config['PATH']}/Dataset"
environ["TRANSFORM_PATH"] = f"{config['PATH']}/Dataset/metadata/transforms"

# Set device
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load and split dataset
handler = DataHandler(batch_size=config["batch_size"])
all_buildings = handler.load_dataset(config["dataset_name"], config["scaler_transform"])
train_buildings = all_buildings[:int(0.8 * len(all_buildings))]
test_buildings = all_buildings[int(0.8 * len(all_buildings)):]

# Train and evaluate all models
for model_class in [NN, RNN, LSTM, GRU]:
    print(f"\n--- Training {model_class.__name__} ---")
    trainer = Trainer(
        model_name=model_class.__name__,
        device=device,
        scaler_transform=config["scaler_transform"],
        activation=config["activation"],
        optimizer_name=config["optimizer_name"],
        lr=config["lr"]
    )
    trainer.train(train_buildings, epochs=config["epochs"])
    _, mae, rmse, r2 = trainer.evaluate(test_buildings)
    print(f"[{model_class.__name__}] MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")


--- Training NN ---
[NN] Epoch 1: Loss = 458.7939
[NN] Epoch 2: Loss = 449.5711
[NN] Epoch 3: Loss = 333.0180
[NN] Epoch 4: Loss = 380.2423
[NN] Epoch 5: Loss = 307.8425
[NN] Epoch 6: Loss = 324.1859
[NN] Epoch 7: Loss = 290.6698
[NN] Epoch 8: Loss = 294.9895
[NN] Epoch 9: Loss = 318.9454
[NN] Epoch 10: Loss = 284.7732
[NN] MAE: 0.2334, RMSE: 0.3677, R²: -0.0458

--- Training RNN ---
[RNN] Epoch 1: Loss = 589.9331
[RNN] Epoch 2: Loss = 558.1313
[RNN] Epoch 3: Loss = 557.0364
[RNN] Epoch 4: Loss = 557.1886
[RNN] Epoch 5: Loss = 555.6446
[RNN] Epoch 6: Loss = 555.3492
[RNN] Epoch 7: Loss = 555.5019
[RNN] Epoch 8: Loss = 555.5766
[RNN] Epoch 9: Loss = 555.6303
[RNN] Epoch 10: Loss = 555.6420
[RNN] MAE: 0.2624, RMSE: 0.4359, R²: -0.4159

--- Training LSTM ---
[LSTM] Epoch 1: Loss = 596.2045
[LSTM] Epoch 2: Loss = 553.0597
[LSTM] Epoch 3: Loss = 529.9367
[LSTM] Epoch 4: Loss = 403.3179
[LSTM] Epoch 5: Loss = 340.8698
[LSTM] Epoch 6: Loss = 301.4939
